In [1]:
import cv2
import numpy as np
import os
from PIL import Image

In [2]:
os.getcwd()

'C:\\Users\\IQ-6'

In [3]:
os.chdir('C:\\Dataset')
os.getcwd()

'C:\\Dataset'

In [4]:
os.listdir('.')

['0',
 '1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9']

In [5]:
#Делаем размер картинки 40 на 40 пкс
def scale(PicName): 
    size = (40, 40)
    resizedPic = cv2.resize(PicName, size, interpolation = cv2.INTER_AREA)
    return resizedPic
#Открытие картинки
def openPicture(PicName):
    image = cv2.imread(PicName)
    return image
#Показ картинки в окне
def showPicture(PicName):
    cv2.imshow("Pik4a", PicName)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
#Делаем картинку черно-белой
def transform_gray(PicName):
    gray_pic = cv2.cvtColor(PicName, cv2.COLOR_BGR2GRAY)
    ret, threshold_image = cv2.threshold(gray_pic, 127, 255, 0)
    return gray_pic

In [12]:
#Делаем все изображения одной переменной
arr1 = []
arr2 = []
for DirName in os.listdir('.'):
    os.chdir(DirName)
    for PicName in os.listdir('.'):
        image = openPicture(PicName)
        image_1 = transform_gray(image)
        image_2 = scale(image_1)
        arr1.append(image_2)
        arr2.append(int(DirName))
    os.chdir('..')
arr1 = np.array(arr1).astype('float32')
arr2 = np.array(arr2).astype('float32')

In [13]:
arr1.shape

(18539, 40, 40)

In [14]:
arr1 /= 255
arr1 = np.array(arr1).astype('float32')
arr1 = arr1.reshape(-1, 40, 40, 1)
arr1.shape

(18539, 40, 40, 1)

In [15]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [17]:
model = Sequential()

model.add(Conv2D(input_shape = (40, 40, 1), filters = 32, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(filters = 32, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D())

model.add(Conv2D(filters = 64, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(Conv2D(filters = 128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(rate = 0.2))
model.add(MaxPooling2D())

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(23, activation='softmax'))













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
import keras
model.compile(loss = 'categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [29]:
Categories = keras.utils.to_categorical(arr2,23)

In [30]:
arr1_train, arr1_test, arr2_train, arr2_test = train_test_split(arr1, Categories, test_size=0.15, random_state=13)

In [31]:
history = model.fit(arr1_train, arr2_train, epochs=3, batch_size=25)

Epoch 1/3
15758/15758 [==============================] - 31s 2ms/step - loss: 0.0168 - acc: 0.9948
Epoch 2/3
15758/15758 [==============================] - 31s 2ms/step - loss: 0.0121 - acc: 0.9974
Epoch 3/3
15758/15758 [==============================] - 31s 2ms/step - loss: 0.0178 - acc: 0.9952


In [26]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 38, 38, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 38, 38, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 19, 19, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 17, 17, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 6, 64)          18496     
__________

In [32]:
model.save('MoyaArchitektura.HDF5')